# Simulator Wrapper

In [1]:
!pip install pyNetLogo
!pip install JPype1
!pip install GPy==1.10.0
!pip install GPyOpt==1.2.1
!pip install emukit

In [2]:
import numpy as np

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home=r"D:\Programs\NetLogo 6.2.0", netlogo_version="6.2")

In [4]:
netlogo.load_model(r'C:\Users\Chengzu\Downloads\Blood_Sugar_Regulation.nlogo')

In [5]:
problem = {
        'num_vars': 4,
        'names': [
                  'metabolic-rate',
                  'insulin-sensitivity',
                  'glucose-sensitivity',
                  'glucagon-sensitivity',
                  ],
        'bounds': [[0, 200],
                   [0., 1.],
                   [0., 1.],
                   [0., 1.]]
    }

In [6]:
def run_simulation(experiment, start_time_step=90, end_time_step=100):
    """ run a netlogo model
    Parameters
    ----------
    experiments : dict
    """

    # Set the input parameters
    
    for key, value in experiment.items():
        if key == 'random-seed':
            # The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))

    netlogo.command('setup')
    # Run for 100 ticks and return the number of glucose
    counts = netlogo.repeat_report(['count glucoses', 'count insulins', 'count glucagons'], 101)

    results = pd.Series([counts['count glucoses'].values[start_time_step: end_time_step].mean()],
                        index=['Avg. glucoses'])
    return results

# Load Specified Parameters

In [7]:
def specify_parameters(file: str, variable_X: list):
    '''
    The file should be a .csv file, with who, metabolic-rate, insulin-sensitivity, glucose-sensitivity, glucagon-sensitivity as its title.
    The returned params are a list full of lists
    '''
    params = []
    import pandas as pd
    df = pd.read_csv(file)
    for i in range(len(df)):
        item = df.iloc[i].to_dict()
        _ = item.pop("who")
        item_values = []
        for k, v in item.items():
            if k not in variable_X:
                continue
            else:
                if k == "metabolic-rate":
                    v = v/200
                item_values.append(v)
        params.append(item_values)
    return params

# Initialize Simulator

In [8]:
class Simulator:
    def __init__(self, variable_X):
        self.variable_X = variable_X
        
    def simulate(self, input_values):
        assert len(input_values[0]) == len(self.variable_X)
        item = {}
        for k, v in zip(self.variable_X, input_values[0]):
            if k == "metabolic-rate":
                v = v * 200
            item[k] = v
        result = run_simulation(item)
        return np.array([result['Avg. glucoses']]).reshape(-1, 1)
    
    def opt_simulate(self, input_values):
        assert len(input_values[0]) == len(self.variable_X)
        item = {}
        for k, v in zip(self.variable_X, input_values[0]):
            if k == "metabolic-rate":
                v = v * 200
            item[k] = v
        result = run_simulation(item)
        return np.array([np.abs(result['Avg. glucoses'] - 4000)]).reshape(-1, 1)

# Gaussian Process as emulator

In [9]:
import GPy

In [10]:
kernel_list = ["RBF", "Matern52", "Matern32", "Exponential", "OU", "RatQuad"]
# https://scikit-learn.org/stable/modules/gaussian_process.html#gp-kernels
# https://gpy.readthedocs.io/en/deploy/GPy.kern.src.html#module-GPy.kern.src.stationary

def get_kernel(kernel_name: str, input_dim: int, length_scale: float, variance: float):
    # squared-exponential
    if kernel_name == "RBF":
        return GPy.kern.RBF(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    # Matern covariance
    elif kernel_name == "Matern52":
        return GPy.kern.Matern52(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "Matern32":
        return GPy.kern.Matern32(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    # # periodic covariance
    elif kernel_name == "Periodic":
        return GPy.kern.Periodic(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "Exponential":
        return GPy.kern.Exponential(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "OU":
        return GPy.kern.OU(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "RatQuad":
        return GPy.kern.RatQuad(input_dim=input_dim, variance=variance, lengthscale=length_scale)

    else:
        raise KeyError("Unsupported kernel here.")

In [11]:
def emulate(method: str, X: list, Y: list, kernel_name: str, length_scale: float, variance: float, noise_var: float = 1e-10, verbose: bool = False):
    if method == "GP":
        model = Gaussian_Process(X, Y, kernel_name, length_scale, variance, verbose, noise_var)
        return model
    else:
        raise ValueError("Unsupported Emulator.")

In [12]:
def Gaussian_Process(X: list, Y: list, kernel_name: str, length_scale: float, variance: float, verbose: bool = False, noise_var: float = 1e-10):
    if verbose:
        print(f"Using {kernel_name} kernel and Gaussian Process as Emulator.")
    input_dim = len(X[0])
    print('input_dim', input_dim)
    kernel = get_kernel(kernel_name, input_dim, length_scale, variance)
    model = GPy.models.GPRegression(np.array(X), np.array(Y)[:,np.newaxis], kernel, noise_var=noise_var) # noise_var=1e-10
    # model.optimize_restarts(num_restarts=10)
    # https://gpy.readthedocs.io/en/devel/GPy.models.html
    _ = model.optimize(messages=verbose) # Optimize parameters of covariance function
    print(_)
    print(f"{kernel_name}-log_likelihood", model.log_likelihood())
    return model

# Bayesian Optimization

In [13]:
def get_acquisition_function(fn, model, space):
    if fn == "ivr":
        from emukit.experimental_design.acquisitions import IntegratedVarianceReduction
        acquisition_function = IntegratedVarianceReduction(model, space)
    elif fn == "ei":
        from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
        acquisition_function = ExpectedImprovement(model=model)
    elif fn == "mv":
        from emukit.experimental_design.acquisitions import ModelVariance
        acquisition_function = ModelVariance(model)
    else:
        raise ValueError("Unsupported Acquisition Function.")
    return acquisition_function

In [14]:
def fc(x: list, alpha: list=[0.4, 0.2, 0.2, 0.2]):
    """
    Constraints function
    """
    c = (x[0] - 0.4/alpha[0])**2 + ((x[1] - 0.4)/alpha[1])**2 + ((x[2] - 0.5) / alpha[2])**2 + ((x[3] - 0.8)/alpha[3])**2 - 2.5
    return np.array(c)

In [15]:
def Bayesian_Optimization(
    evaluation_step: int, 
    X: list,
    Y: list,
    variable_X: list, 
    kernel_n: str,
    length_scale: float,
    variance: float,
    acquisition_fn: str,
    simulator
    ):
    if "metabolic-rate" in variable_X:
        meta_idx = variable_X.index("metabolic-rate")
    else:
        meta_idx = -1

    from emukit.model_wrappers import GPyModelWrapper
    from emukit.core import ParameterSpace, ContinuousParameter
    from emukit.core.constraints import NonlinearInequalityConstraint

    s = []
    x_plot = []
    for var in variable_X:
        if var in ["insulin-sensitivity"]:
            x_p = np.linspace(0.2, 0.7, 100)
            p = ContinuousParameter(var, 0.2, 0.7)
        elif var in ["glucose-sensitivity"]:
            x_p = np.linspace(0.4, 0.7, 100)
            p = ContinuousParameter(var, 0.4, 0.7)
        elif var in ["glucagon-sensitivity"]:
            x_p = np.linspace(0.6, 0.9, 100)
            p = ContinuousParameter(var, 0.6, 0.9)
        else:
            x_p = np.linspace(0.3, 0.6, 100)
            p = ContinuousParameter(var, 0.3, 0.6)
        s.append(p)
        x_plot.append(x_p)

    space = ParameterSpace(s)
    print("Bounds: ", space.get_bounds())
    space.constraints = [NonlinearInequalityConstraint(
        constraint_function=fc,
        lower_bound=np.array([-3]),
        upper_bound=np.array([0])
    )]
    
    model = emulate(
      method="GP",
      X=X,
      Y=Y,
      kernel_name=kernel_n,
      length_scale=length_scale,
      variance=variance
    )
    emukit_model = GPyModelWrapper(model)
    
    acquisition_function = get_acquisition_function(acquisition_fn, emukit_model, space)
    
    from emukit.core.optimization import GradientAcquisitionOptimizer

    # Create acquisition optimizer with constraints
    acquisition_optimizer = GradientAcquisitionOptimizer(space)
    
    from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
    bayesopt_loop = BayesianOptimizationLoop(
        model = emukit_model,
        space = space,
        acquisition = acquisition_function,
        acquisition_optimizer=acquisition_optimizer,
        batch_size = 1)
    
    if len(variable_X) == 2:
        bayesopt_loop.iteration_end_event.append(plot_progress)
    
    max_iterations = evaluation_step
    bayesopt_loop.run_loop(simulator.opt_simulate, max_iterations)
    
    results = bayesopt_loop.get_results()
    return results, bayesopt_loop

# Experiments

In [16]:
variable_X = ["metabolic-rate", "insulin-sensitivity", "glucose-sensitivity", "glucagon-sensitivity"]
simulator = Simulator(variable_X)

In [17]:
X = specify_parameters(
    file=r"../my_sample_data.csv",
    variable_X=variable_X
)

In [18]:
Y = [simulator.simulate([p]).reshape(-1)[0] for p in X]

In [19]:
assert len(Y) == len(X)

In [20]:
Y_select = np.abs(np.array(Y) - 4000)
results, model_0 = Bayesian_Optimization(
    evaluation_step=10, 
    X=X,
    Y=Y_select,
    variable_X=variable_X, 
    kernel_n="RBF",
    length_scale=1,
    variance=1,
    acquisition_fn="ei",
    simulator=simulator
)

Bounds:  [(0.3, 0.6), (0.2, 0.7), (0.4, 0.7), (0.6, 0.9)]
input_dim 4
Optimizer: 				 L-BFGS-B (Scipy implementation)
f(x_opt): 				 78.677
Number of function evaluations: 	 86
Optimization status: 			 Converged
Time elapsed: 				 0:00:00.107470

RBF-log_likelihood -78.67685208605414


 D:\Programs\Anaconda3\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1
 D:\Programs\Anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning:delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.


In [21]:
model_0.loop_state.X

array([[0.48      , 0.55      , 0.56      , 0.69      ],
       [0.39      , 0.25      , 0.47      , 0.73      ],
       [0.33      , 0.59      , 0.58      , 0.75      ],
       [0.53      , 0.32      , 0.47      , 0.81      ],
       [0.455     , 0.44      , 0.58      , 0.79      ],
       [0.415     , 0.31      , 0.43      , 0.87      ],
       [0.345     , 0.54      , 0.51      , 0.77      ],
       [0.4       , 0.22      , 0.59      , 0.67      ],
       [0.56      , 0.51      , 0.55      , 0.72      ],
       [0.355     , 0.49      , 0.43      , 0.69      ],
       [0.6       , 0.62406366, 0.7       , 0.85827072],
       [0.59999852, 0.28338516, 0.69999922, 0.60000246],
       [0.6       , 0.6814158 , 0.5663713 , 0.89999998],
       [0.59999999, 0.60105643, 0.69999999, 0.68521205],
       [0.59999999, 0.62232872, 0.69999997, 0.73542613],
       [0.59999996, 0.51683585, 0.69999995, 0.60012662],
       [0.59999986, 0.20000127, 0.69999991, 0.89999931],
       [0.3       , 0.55194237,

In [22]:
model_0.loop_state.Y

array([[1530.2],
       [3820.7],
       [1712.6],
       [3414.5],
       [1789.4],
       [4468.9],
       [2359.6],
       [2066.8],
       [1446.2],
       [3478.4],
       [ 386.6],
       [ 694.2],
       [1614.1],
       [ 137.7],
       [ 274.5],
       [ 266.5],
       [1483.6],
       [ 683.4],
       [ 794.6],
       [2003.1]])

In [23]:
results.minimum_value

137.69999999999982

In [24]:
results.minimum_location

array([0.59999999, 0.60105643, 0.69999999, 0.68521205])